In [1]:
using TensorKit
import TensorKit.×
include("./iPEPS/iPEPS.jl")
include("./iPEPS/swap_gate.jl")
include("./iPEPS/util.jl")

inv

In [ ]:
## follow convention (l, t, σ, r, b)
## domain: r, b
## codomain: l, t, σ
M11 = TensorMap(randn, ComplexF64, ℂ^3 ⊗ ℂ^3 ⊗ ℂ^2, ℂ^3 ⊗ ℂ^3)
@show domain(M11)
@show space(M11)
M12 = TensorMap(randn, ComplexF64, ℂ^3 ⊗ ℂ^3 ⊗ ℂ^2, ℂ^3 ⊗ ℂ^3)

## contraction 后用; 分割 (codomain; domain), 原来的domain 放到 codomain，或反过来，空间都要变为对偶空间（加'）
@tensor M1112[l1, t1, σ1, σ2, b1; t2, r2, b2] := M11[l1, t1, σ1, r1, b1] * M12[r1, t2, σ2, r2, b2]
# B = permute(A, (3,), (1, 2, 4, 5))
# @show domain(B)
@show domain(M1112)
@show codomain(M1112)

In [2]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet
# @show typeof(pspace)
# @show GradedSpace isa ElementarySpace
# @show pspace isa VectorSpace
# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)

Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)

In [ ]:
A = id(pspace)
@show A
exp!(A)
@show A
nothing

In [3]:
include("./simple_update/iPEPS_GammaLambda.jl")
ipeps = iPEPSΓΛ(pspace, aspacel, aspacet, 2, 2; dtype=ComplexF64)

gateNN = TensorMap(randn, ComplexF64, pspace ⊗ pspace, pspace ⊗ pspace)

nothing

In [4]:
xx = 1
yy = 1
Dk = 5
swgate = swap_gate(space(ipeps[xx, yy+1].Γ)[3]', space(ipeps[xx, yy+1].Γ)[1]')
@tensor Γu[l, t, p; r, b] := ipeps[xx, yy].Γ[le, te, p, re, b] * ipeps[xx, yy].t[t, te] * ipeps[xx, yy].r[re, r] * ipeps[xx, yy].l[l, le]
@tensor Γd[l, t, p; r, b] := ipeps[xx, yy+1].Γ[le, t, pin, re, be] * ipeps[xx, yy+1].l[lin, le] * ipeps[xx, yy+1].r[re, r] *
                             ipeps[xx, yy+1].b[be, b] * swgate[pin, lin, p, l]
# 两次QR
Xu, vu = leftorth(Γu, ((1, 2, 4), (3, 5)))
wd, Yd = rightorth(Γd, ((2, 3), (1, 4, 5)))
# 作用 Trotter Gate
@tensor mid[toX, pu, pd; toY] := vu[toX, puin, toV] * ipeps[xx, yy].b[toV, toW] * gateNN[pu, pd, puin, pdin] *
                                 wd[toW, pdin, toY]
vnew, λnew, wnew, err = tsvd(mid, ((1, 2), (3, 4)); trunc=truncdim(Dk))
swgate = swap_gate(space(wd)[2]', space(Yd)[2]')
@tensor Γdnew[l, t, p; r, b] := Yd[toY, lein, re, be] * wnew[t, pin, toY] * swgate[pin, lein, p, le] *
                                inv(ipeps[xx, yy+1].l)[l, le] * inv(ipeps[xx, yy+1].r)[re, r] * inv(ipeps[xx, yy+1].b)[be, b]                              
@tensor Γunew[l, t, p; r, b] := Xu[le, te, re, toX] * vnew[toX, p, b] * inv(ipeps[xx, yy].t)[t, te] *
                                inv(ipeps[xx, yy].r)[re, r] * inv(ipeps[xx, yy].l)[l, le]
# 更新 tensor
# ipeps[xx, yy].b = λnew
# ipeps[xx, yy+1].t = λnew
# ipeps[xx, yy].Γ = Γunew
# ipeps[xx, yy+1].Γ = Γdnew

nothing

In [9]:
# @show space(ipeps[xx, yy].r)
# @show space(ipeps[2, 2].r)
# @show space(ipeps[xx, yy].l)
# @assert ipeps[xx+1, yy].r == ipeps[xx, yy].l
# @show space(Yr)[3]
# @show space(Γrnew)[4]
# @show space(ipeps[xx+1, yy].r)[1]
swgate = swap_gate(space(Γunew)[3]', space(Γunew)[1]')

@tensor test[lin, t, pin; re, be] := Γunew[le, t, pin, re, be] * ipeps[1, 1].l[lin, le]
@tensor test2[lin, t, pin; r, be] := test[lin, t, pin; re, be] * ipeps[1, 1].r[re, r]
@tensor test3[lin, t, pin; r, b] := test2[lin, t, pin; r, be] * λnew[be, b]
@tensor test4[l, t, p; r, b] := test3[lin, t, pin; r, b] * swgate[pin, lin, p, l]
# @tensor test[le, te, p; r, be] := Γrnew[le, te, p, re, be] *  ipeps[xx+1, yy].r[re, r]
@show space(test4)                
nothing

space(test4) = (Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)) ← (Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1))
